In [1]:
import pandas as pd

In [3]:
pd.read_csv("../data/Wuhan-2019-nCoV.csv")

,date,country,countryCode,province,provinceCode,city,cityCode,confirmed,suspected,cured,dead
0,2019-12-01,中国,CN,NaN,NaN,NaN,NaN,1,0,0,0
1,2019-12-01,中国,CN,湖北省,420000.0,NaN,NaN,1,0,0,0
2,2019-12-01,中国,CN,湖北省,420000.0,武汉市,420100.0,1,0,0,0
3,2019-12-02,中国,CN,NaN,NaN,NaN,NaN,1,0,0,0
4,2019-12-02,中国,CN,湖北省,420000.0,NaN,NaN,1,0,0,0
5,2019-12-02,中国,CN,湖北省,420000.0,武汉市,420100.0,1,0,0,0
6,2019-12-03,中国,CN,NaN,NaN,NaN,NaN,1,0,0,0
7,2019-12-03,中国,CN,湖北省,420000.0,NaN,NaN,1,0,0,0
8,2019-12-03,中国,CN,湖北省,420000.0,武汉市,420100.0,1,0,0,0
9,2019-12-04,中国,CN,NaN,NaN,NaN,NaN,1,0,0,0
